In [1]:
import os 
import re
import matplotlib.pyplot as plt
import rmgpy.chemkin
import numpy as np
import cantera as ct
import random
import pandas as pd
from rmgpy.chemkin import load_chemkin_file
%matplotlib inline

Flipping one by one, siddhas CH3F model and CH3F model (both with original thermo orders)

In [2]:
#load the chemkin functions
def compare_rxns(model_1_reactions, model_2_reactions): 
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

    ################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (model_1_reactions, model_2_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    if len(to_remove)!=0:
        for reactionList, reaction in to_remove:
            reactionList.remove(reaction)
        print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = []
    common_reactions_equations = []
    unique_reactions_1 = []
    unique_reactions_2 = []
    copy_of_model_1_reactions = model_1_reactions.copy() #list 1
    copy_of_model_2_reactions = model_2_reactions.copy() #list 2
    
    
    # Reaction Diff
    for i, _1_rxn in enumerate(copy_of_model_1_reactions):
        for j, _2_rxn in enumerate(copy_of_model_2_reactions):
            if _1_rxn.is_isomorphic(_2_rxn):
                common_reactions.append([_1_rxn, _2_rxn])
                break
    print(f'{len(common_reactions)} common reactions')
   
    #find the unique reactions of each model
    for model_2_rxn in copy_of_model_2_reactions:
        for r1, r2 in common_reactions:
            #if model_2_rxn is r2:# and type(model_2_rxn.kinetics)==type(r2.kinetics):
            if model_2_rxn.is_isomorphic(r2):
                break
        else:
            unique_reactions_2.append(model_2_rxn)
    for model_1_rxn in copy_of_model_1_reactions:
        for r1, r2 in common_reactions:
            #if model_1_rxn is r1 and type(model_1_rxn.kinetics)==type(r1.kinetics):
            if model_1_rxn.is_isomorphic(r1):
                break
        else:
            unique_reactions_1.append(model_1_rxn)
            
        
    
    #find the reactions that have different kinetics (aren't identical and aren't similar)
    
    different_kinetics_reactions = []
    identical_kinetics_reactions = []
    similar_kinetics_reactions = []
    
    for r1, r2 in common_reactions: 
        if (r2.kinetics.is_identical_to(r1.kinetics)==False) and (r2.kinetics.is_similar_to(r1.kinetics)==False):
            different_kinetics_reactions.append((r1, r2))
        if r2.kinetics.is_identical_to(r1.kinetics)==True:
            identical_kinetics_reactions.append((r1, r2))
        if r2.kinetics.is_similar_to(r1.kinetics)==True:
            similar_kinetics_reactions.append((r1, r2))
            
            
    return similar_kinetics_reactions, identical_kinetics_reactions, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactions_1, unique_reactions_2
    
       
def compare_species(model_1_species_list, model_2_species_list):
    """ Compares species list between models. """
    
    #two lists of same species, just have different names in each model
    common_species_1 = []
    common_species_2 = []
    
    #combined list of common species, with tuples of the two different names
    common_species = []
    
    for spc_1 in model_1_species_list:
        for spc_2 in model_2_species_list:
            if spc_1.is_isomorphic(spc_2):
                common_species_1.append(spc_1)
                common_species_2.append(spc_2)
                common_species.append((spc_1, spc_2))
                
                
    unique_species_1 = [spc_1 for spc_1 in model_1_species_list if spc_1 not in common_species_1]
    unique_species_2 = [spc_2 for spc_2 in model_2_species_list if spc_2 not in common_species_2]
    print(f'{len(common_species)} common species')


                
    return common_species, unique_species_1, unique_species_2

In [7]:
#full path
full_path = '/work/westgroup/nora/Code/'

#Siddha's improvements (thermos in original order) #107 species
siddhas_chmkn = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddha_thermos_original_order/chemkin/chem_annotated.inp'
siddhas_transport = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddha_thermos_original_order/chemkin/tran.dat'
siddhas_species = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddha_thermos_original_order/chemkin/species_dictionary.txt'

#original CH3F based on David's general input.py setup 
CH3F_chmkn = full_path + 'projects/halogens/refrigerants/blends/redos_for_march_conference/singles/CH3F/chemkin/chem_annotated.inp'
CH3F_transport = full_path + 'projects/halogens/refrigerants/blends/redos_for_march_conference/singles/CH3F/chemkin/tran.dat'
CH3F_species = full_path + 'projects/halogens/refrigerants/blends/redos_for_march_conference/singles/CH3F/chemkin/species_dictionary.txt'

#thermos rearranged #114 species 

thermos_rearranged_chmkn = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/thermos_rearranged/3rd_try/chemkin/chem_annotated.inp'
thermos_rearranged_transport = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/thermos_rearranged/3rd_try/chemkin/tran.dat'
thermos_rearranged_species = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/thermos_rearranged/3rd_try/chemkin/species_dictionary.txt'

#siddhas with thermo orders rearranged #111 species
siddhas_thermos_rearranged_chmkn = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddha_thermos_rearranged/4th_try/chemkin/chem_annotated.inp'
siddhas_thermos_rearranged_transport = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddha_thermos_rearranged/4th_try/chemkin/tran.dat'
siddhas_thermos_rearranged_species = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddha_thermos_rearranged/4th_try/chemkin/species_dictionary.txt'

#siddhas with kinetic orders switched 
siddhas_kinetics_rearranged_chmkn = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddhas_halogen_pdep_fluoromethane_switched/chemkin/chem_annotated.inp'
siddhas_kinetics_rearranged_transport = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddhas_halogen_pdep_fluoromethane_switched/chemkin/tran.dat'
siddhas_kinetics_rearranged_species = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/Siddhas_halogen_pdep_fluoromethane_switched/chemkin/species_dictionary.txt'

#load siddas
species_siddhas, reactions_siddhas = load_chemkin_file(siddhas_chmkn, dictionary_path=siddhas_species, transport_path=siddhas_transport)

#load previous CH3F
species_CH3F, reactions_CH3F = load_chemkin_file(CH3F_chmkn, dictionary_path=CH3F_species, transport_path=CH3F_transport)

#load thermos rearranged 
species_thermos_rearranged, reactions_thermos_rearranged = load_chemkin_file(thermos_rearranged_chmkn, dictionary_path=thermos_rearranged_species, transport_path=thermos_rearranged_transport)

#load siddhas with thermo rearranged
species_siddhas_thermos_rearranged, reactions_siddhas_thermos_rearranged = load_chemkin_file(siddhas_thermos_rearranged_chmkn, dictionary_path=siddhas_thermos_rearranged_species, transport_path=siddhas_thermos_rearranged_transport)

#load siddhas kinetic orders switched 
species_siddhas_kinetics_rearranged, reactions_siddhas_kinetics_rearranged = load_chemkin_file(siddhas_kinetics_rearranged_chmkn, dictionary_path=siddhas_kinetics_rearranged_species, transport_path=siddhas_kinetics_rearranged_transport)


In [58]:
#comparison of siddhas additions (with original thermo order) and previous CH3F model 
model_1_species_list=species_siddhas
model_2_species_list=species_CH3F
model_1_reactions= reactions_siddhas
model_2_reactions= reactions_CH3F


similar_kinetics_reactions, identical_kinetics_reactions, different_kinetics_reactions, common_reactions_siddha, common_reactions_equations, unique_reactions_siddhas, unique_reactions_previous = compare_rxns(model_1_reactions, model_2_reactions)
common_species_siddha, unique_species_siddhas, unique_species_previous = compare_species(model_1_species_list, model_2_species_list)

1165 common reactions
74 common species


In [9]:
print(len(common_species), len(common_reactions), len(model_2_reactions), len(model_1_reactions), len(unique_reactions_previous), len(unique_reactions_siddhas))
#model 2 previous CH3F, model 1 is siddhas

74 1165 1440 2126 347 961


## Flipping

In [10]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
#import numpy as np
import subprocess
import csv
#import scipy
import copy
import os

In [11]:
def switch_kinetics(test_rxn_1):
    rxn_2_edited = copy.deepcopy(test_rxn_1)
    rxn_2_edited.kinetics

    new_reactants = []
    print('replacing')
    for spc in rxn_2_edited.reactants:
        print(spc)
        for k,v in _1_to_2_dictionary.items():
            if k.is_identical(spc):
                new_reactants.append(v)
    print('with')
    for i in new_reactants:
        print(i)
    rxn_2_edited.reactants = new_reactants

    new_products = []
    print('replacing')
    for spc in rxn_2_edited.products:
        print(spc)
        for k,v in _1_to_2_dictionary.items():
            if k.is_identical(spc):
                new_products.append(v)
    print('with')
    for i in new_products:
        print(i)
    rxn_2_edited.products = new_products
    
    #now take care of the flux pairs
    new_pairs=[]
    for pairs in rxn_2_edited.pairs: 
        new_pair = []
        for spc in pairs: 
            for k,v in _1_to_2_dictionary.items():
                if k.is_identical(spc):
                    new_pair.append(v)
        new_pairs.append(new_pair)
        
    rxn_2_edited.pairs=new_pairs               
            
    
    return rxn_2_edited

In [12]:
to_remove = []
for reactionList in (model_1_reactions, model_2_reactions):
    for reaction in reactionList:
        for side in (reaction.products, reaction.reactants):
            for species in side:
                if not species.molecule:
                    to_remove.append((reactionList, reaction))
                    print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                    break
if len(to_remove)!=0:
    for reactionList, reaction in to_remove:
        reactionList.remove(reaction)
    print(f'Following reactions removed because they have unknown species: {to_remove}')
    
copy_of_model_1_reactions = model_1_reactions.copy() #list 1
copy_of_model_2_reactions = model_2_reactions.copy() #list 2

#lets chunk together the dups of each model
model_1_reactions_dups_recognized = [] #will be a list of lists. Each list inside will have either 1 rxn (if no dup) or multiple if there are dups
model_2_reactions_dups_recognized = [] #same as above

#need to chunk together reactions in model 1 
for reaction in copy_of_model_1_reactions: 
    was_a_dup=False
    #go through all the lists of rxns
    for list_of_rxns in model_1_reactions_dups_recognized: 
        #just need to check the first one
        if list_of_rxns[0].is_isomorphic(reaction):
            list_of_rxns.append(reaction) 
            #print('was a dup')
            was_a_dup=True
            
    if was_a_dup==False:
        #print('not a dup')
        model_1_reactions_dups_recognized.append([reaction])
        
count_1=0
for i in model_1_reactions_dups_recognized: 
    if len(i)>1:
        for x in i:
            #print(x)
            count_1+=1
        #print('\n')
        
    if len(i)==1: 
        #print(i[0])
        # print('\n')
        count_1+=1
        
#did we get all the reactions in model 1?
assert count_1==len(model_1_reactions), "You did not catch all the reactions in model 1!"


#need to chunk together reactions in model 2 
for reaction in copy_of_model_2_reactions: 
    was_a_dup=False
    #go through all the lists of rxns
    for list_of_rxns in model_2_reactions_dups_recognized: 
        #just need to check the first one
        if list_of_rxns[0].is_isomorphic(reaction):
            list_of_rxns.append(reaction) 
            was_a_dup=True
            
    if was_a_dup==False:
        model_2_reactions_dups_recognized.append([reaction])
        
count_2=0
for i in model_2_reactions_dups_recognized: 
    if len(i)>1:
        for x in i:
            #print(x)
            count_2+=1
        #print('\n')
        
    if len(i)==1: 
        #print(i[0])
        # print('\n')
        count_2+=1
        
#did we get all the reactions in model 2?
assert count_2==len(model_2_reactions), "You did not catch all the reactions in model 2!"


#now let's find the common reactions: 
common_reactions_with_dups_grouped = []

for rxn_list_2 in model_2_reactions_dups_recognized:
        rxn_2 = rxn_list_2[0]
        
        for rxn_list_1 in model_1_reactions_dups_recognized: # make a copy so you don't remove from the list you are iterating over
            rxn_1 = rxn_list_1[0]
            
            if rxn_2.is_isomorphic(rxn_1):
                common_reactions_with_dups_grouped.append((rxn_list_1, rxn_list_2)) 

print(f'Here is the actual number of common reactions (with duplicate reactions grouped): {len(common_reactions_with_dups_grouped)}')

Here is the actual number of common reactions (with duplicate reactions grouped): 1093


In [13]:
print(f'Here is actual number of all reactions with duplicates grouped: model 1: {len(model_1_reactions_dups_recognized)}, model 2: {len(model_2_reactions_dups_recognized)}' )

Here is actual number of all reactions with duplicates grouped: model 1: 1969, model 2: 1359


In [14]:
_1_to_2_dictionary = {}
for spc_1, spc_2 in common_species:
    _1_to_2_dictionary[spc_1]=spc_2

In [ ]:
#test for flipping

In [17]:
#TEST
#this will have the kinetics of rxn_1 but will have the species of rxn_2_
[test_rxn_1],[test_rxn_2] = common_reactions_with_dups_grouped[181]
print(type(test_rxn_1))
print(test_rxn_1.kinetics)

[test_rxn_3],[test_rxn_4] = common_reactions_with_dups_grouped[0]
print(type(test_rxn_3))
print(test_rxn_3.kinetics)

new_reaction = switch_kinetics(test_rxn_1)
print(str(new_reaction))
print(new_reaction.kinetics)
for pairs in new_reaction.pairs:
    for spc in pairs:
        print(spc)

#works on none pdep reactions! if you replace [0] with [181] (which is a pdep reaction), it wont work :( 

<class 'rmgpy.rmg.pdep.PDepReaction'>
Chebyshev(coeffs=[[8.705,1.999,-0.0003783,-0.0002099],[0.09638,0.0002107,0.0001466,8.135e-05],[-0.03536,6.206e-05,4.319e-05,2.397e-05],[-0.01905,9.245e-06,6.435e-06,3.574e-06],[-0.005078,-5.441e-06,-3.784e-06,-2.098e-06],[-0.006684,1.042e-06,7.255e-07,4.029e-07]], kunits='cm^3/(mol*s)', Tmin=(300,'K'), Tmax=(2500,'K'), Pmin=(0.01,'atm'), Pmax=(98.692,'atm'))
<class 'rmgpy.data.kinetics.library.LibraryReaction'>
Arrhenius(A=(9.841e+13,'cm^3/(mol*s)'), n=0, Ea=(15.31,'kcal/mol'), T0=(1,'K'))
replacing
O(7)
OH(5)
with
O(6)
OH(7)
replacing
HO2(11)
with
HO2(11)
O(6) + OH(7) <=> HO2(11)
Chebyshev(coeffs=[[8.705,1.999,-0.0003783,-0.0002099],[0.09638,0.0002107,0.0001466,8.135e-05],[-0.03536,6.206e-05,4.319e-05,2.397e-05],[-0.01905,9.245e-06,6.435e-06,3.574e-06],[-0.005078,-5.441e-06,-3.784e-06,-2.098e-06],[-0.006684,1.042e-06,7.255e-07,4.029e-07]], kunits='cm^3/(mol*s)', Tmin=(300,'K'), Tmax=(2500,'K'), Pmin=(0.01,'atm'), Pmax=(98.692,'atm'))
O(6)
HO2(11)


In [19]:
total=0
for list in model_2_reactions_dups_recognized:
    for x in list:
        total+=1
total

1440

In [78]:
#let's do the actual flipping now 
############################### one by one, lets flip all the common reactions in the RMG model to siddhas kinetics##########################################
#start with all the reactions in the original RMG model: 
#model_2_reactions= reactions_CH3F

count=0
for tup in common_reactions_with_dups_grouped: 
    print('starting_new')
    worth_switching=True
    
    rxns_to_be_included = [] #will be a list of lists
    
    rxns_1_from_tuple, rxns_2_from_tuple = tup[0], tup[1] #these are each lists
    
    for ind, rxns_2 in enumerate(model_2_reactions_dups_recognized): 
        if worth_switching==False: 
            continue 
        if rxns_2 == rxns_2_from_tuple:
            # print(ind, 'identified match')
            #print(f'Found the match for common reaction in \'model_2_reactions_dups_recognized\' with index {ind}!')
            count+=1 #increment count to make sure we have got this common reaction)
            
            #if the lengths are the same, see if the kinetics are identical
            if len(rxns_1_from_tuple)==len(rxns_2_from_tuple): 
                new_total_rxn_count = len(model_2_reactions)
                
                #if its only a one to one pairing, then see if the kinetics are the same
                if len(rxns_1_from_tuple)==1:
                    i=rxns_1_from_tuple[0]
                    j=rxns_2_from_tuple[0]
                    if not i.kinetics.is_identical_to(j.kinetics):
                        # print(ind, "Ooooo, difference in these 1:1 kinetics!")
                        switched_rxn = switch_kinetics(i)
                        rxns_to_be_included.append([switched_rxn])
                        print(ind,'switched')
                        file_name=ind
                        worth_switching=True #flag
                    else:
                        worth_switching=False
                        #print("not switching, these 1:1 kinetics were the same")
                
                if len(rxns_1_from_tuple)>1:
                    switched_dups = []
                    
                    for rxn_1 in rxns_1_from_tuple:

                        switched_rxn = switch_kinetics(rxn_1)
                        print('switched')
                        switched_dups.append(switched_rxn)
                        file_name=ind

                    worth_switching=True #flag
                    rxns_to_be_included.append(switched_dups) #This way the dups are still grouped
                    
                              
            if worth_switching==False:
                break          
                    
            if len(rxns_1_from_tuple)!=len(rxns_2_from_tuple):
                switched_dups = []
                print(f'duplicates of unequal lengths! {len(rxns_1_from_tuple)} & {len(rxns_2_from_tuple)}')
                for rxn_1 in rxns_1_from_tuple:
                    print("Ooooo, difference in these uneven rxn list!")
                    # try: 
                    switched_rxn = switch_kinetics(rxn_1)
                    switched_dups.append(switched_rxn)
                    print('switched')
                    rxns_to_be_included.append(switched_dups)
                    file_name=ind
                    # except TypeError: 
                    # print('assume switched')
                                      
                worth_switching=True #flag
                    
                new_total_rxn_count = len(model_2_reactions)- len(rxns_2_from_tuple) + len(rxns_1_from_tuple)
            
                        
        if rxns_2 != rxns_2_from_tuple: 
            # print(ind, 'not a match')
            #add without switching kinetics 
            rxns_to_be_included.append(rxns_2)  
            
        if worth_switching==False:
            break



    if worth_switching==True:
        
        total_rxns_for_new_chemkin = []
        for lists in rxns_to_be_included:
            for x in lists:
                total_rxns_for_new_chemkin.append(x)
        print(len(total_rxns_for_new_chemkin),new_total_rxn_count)
        print('moving on?')
        if len(total_rxns_for_new_chemkin)==new_total_rxn_count:
            print('yayyyy, moving on!')

            #save the chemkin file 

            handpicked_dir = '/work/westgroup/nora/Code/projects/PFAS/models/testing_Siddhas_chemistry/CH3F/flipping/flipped_models'
            chemkin_file = os.path.join(handpicked_dir, f'{file_name}.inp')
            rmgpy.chemkin.save_chemkin_file(chemkin_file, model_2_species_list , total_rxns_for_new_chemkin, verbose=True, check_for_duplicates=True)
            print('file saving chemkin')


assert count==len(common_reactions_with_dups_grouped), "Not all the new chemkin files were generated!" 

#     #save the chemkin file 

#     handpicked_dir = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_with_BROH_specific_species_removed'


#     chemkin_file = os.path.join(handpicked_dir, f'{file_name}.inp')
#     rmgpy.chemkin.save_chemkin_file(chemkin_file, _1_species_list, new_reaction_list, verbose=True, check_for_duplicates=True)



starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new
starting_new

file saving chemkin
starting_new
replacing
H(6)
CH3F(1)
with
H(5)
CH3F(1)
replacing
H2(8)
CH2F(37)
with
H2(8)
CH2F(45)
238 switched
1440 1440
moving on?
yayyyy, moving on!


file saving chemkin
starting_new
replacing
O(7)
CH3F(1)
with
O(6)
CH3F(1)
replacing
OH(5)
CH2F(37)
with
OH(7)
CH2F(45)
239 switched
1440 1440
moving on?
yayyyy, moving on!


file saving chemkin
starting_new
replacing
OH(5)
CH3F(1)
with
OH(7)
CH3F(1)
replacing
H2O(3)
CH2F(37)
with
H2O(3)
CH2F(45)
240 switched
1440 1440
moving on?
yayyyy, moving on!


file saving chemkin
starting_new
replacing
H2O2(12)
CH2F(37)
with
H2O2(12)
CH2F(45)
replacing
HO2(11)
CH3F(1)
with
HO2(11)
CH3F(1)
241 switched
1440 1440
moving on?
yayyyy, moving on!


...

file saving chemkin
starting_new
starting_new
starting_new
replacing
FOH(44)
CHF2(39)
with
OF(133)
CHF2(81)
replacing
OH(5)
CHF3(41)
with
OH(7)
CHF3(41)
1345 switched
1440 1440
moving on?
yayyyy, moving on!


file saving chemkin
starting_new
starting_new
starting_new
starting_new
replacing
F(42)
O=[C]O(144)
with
F(37)
O=[C]O(145)
replacing
FOH(44)
CO(13)
with
OF(133)
CO(13)
1352 switched
1440 1440
moving on?
yayyyy, moving on!


file saving chemkin
starting_new
replacing
H(6)
[O]C(=O)F(152)
with
H(5)
[O]C(=O)F(148)
replacing
FOH(44)
CO(13)
with
OF(133)
CO(13)
1353 switched
1440 1440
moving on?
yayyyy, moving on!


file saving chemkin
starting_new
replacing
F(42)
[O]C=O(141)
with
F(37)
[O]C=O(146)
replacing
FOH(44)
CO(13)
with
OF(133)
CO(13)
1354 switched
1440 1440
moving on?
yayyyy, moving on!


file saving chemkin
starting_new
starting_new
starting_new
starting_new


In [122]:
#test flipped_models 

for ind, rxns_list in enumerate(model_2_reactions_dups_recognized):
    # print(ind, len(rxns_list))
    if ind==240:
        #for x in rxns_list[0].pairs:
        
        rxn_of_interest_1 = rxns_list[0]
        # rxn_of_interest_2 = rxns_list[1]
        print('we want this: ')
        print(rxn_of_interest_1)
        print(rxn_of_interest_1.kinetics)
        print('and')
        # print(rxn_of_interest_2)
        # print(rxn_of_interest_2.kinetics)

for i,j in common_reactions_siddha: 
    if j.is_isomorphic(rxn_of_interest_1): #or j.is_isomorphic(rxn_of_interest_2) :
        #this is the kinetics that it should be switched to!!!
        print('to look like this, but with species of above: ')
        print(i, j)
        print(i.kinetics)
        


#works!


we want this: 
OH(7) + CH3F(1) <=> H2O(3) + CH2F(45)
Arrhenius(A=(1339.5,'cm^3/(mol*s)'), n=3.172, Ea=(0.844,'kcal/mol'), T0=(1,'K'), comment="""Fitted to 50 data points; dA = *|/ 1.06757, dn = +|- 0.00859005, dEa = +|- 0.0467467 kJ/molMatched reaction 3451 HO + CH3F <=> H2O + CH2F in H_Abstraction/training
This reaction matched rate rule [Root_N-3R->F_N-3BrCClHINOS->C_Ext-1R-R_Sp-4R!H-1R_N-3BrClHINOS->Cl_N-1R->S_N-4R!H->Cl_N-3BrHINOS->S_N-3BrHINO->I_N-4B rCFNOS->O_4BrCFNS-u0_N-4BrCFNS->C_N-3BrHNO->Br_3HNO-u1_3HNO->O_4BrFNS->F]
family: H_Abstraction""")
and
to look like this, but with species of above: 
OH(5) + CH3F(1) <=> H2O(3) + CH2F(37) OH(7) + CH3F(1) <=> H2O(3) + CH2F(45)
Arrhenius(A=(2.36e+06,'cm^3/(mol*s)'), n=2.01, Ea=(1.862,'kcal/mol'), T0=(1,'K'))


In [55]:
for i, j in common_reactions: 
    print(i,j)

O2(2) + H(5) <=> O(6) + OH(7) O2(2) + H(5) <=> O(6) + OH(7)
O(6) + H2(8) <=> H(5) + OH(7) O(6) + H2(8) <=> H(5) + OH(7)
OH(7) + H2(8) <=> H(5) + H2O(3) OH(7) + H2(8) <=> H(5) + H2O(3)
OH(7) + OH(7) <=> O(6) + H2O(3) OH(7) + OH(7) <=> O(6) + H2O(3)
H2(8) <=> H(5) + H(5) H2(8) <=> H(5) + H(5)
Ar(9) + H2(8) <=> Ar(9) + H(5) + H(5) Ar(9) + H2(8) <=> Ar(9) + H(5) + H(5)
He(10) + H2(8) <=> He(10) + H(5) + H(5) He(10) + H2(8) <=> He(10) + H(5) + H(5)
O(6) + O(6) <=> O2(2) O(6) + O(6) <=> O2(2)
Ar(9) + O(6) + O(6) <=> Ar(9) + O2(2) Ar(9) + O(6) + O(6) <=> Ar(9) + O2(2)
He(10) + O(6) + O(6) <=> He(10) + O2(2) He(10) + O(6) + O(6) <=> He(10) + O2(2)
O(6) + H(5) <=> OH(7) O(6) + H(5) <=> OH(7)
H2O(3) <=> H(5) + OH(7) H2O(3) <=> H(5) + OH(7)
H2O(3) + H2O(3) <=> H(5) + OH(7) + H2O(3) H2O(3) + H2O(3) <=> H(5) + OH(7) + H2O(3)
H(5) + HO2(11) <=> O2(2) + H2(8) H(5) + HO2(11) <=> O2(2) + H2(8)
H(5) + HO2(11) <=> OH(7) + OH(7) H(5) + HO2(11) <=> OH(7) + OH(7)
H(5) + HO2(11) <=> O(6) + H2O(3) H(5) + HO2(

In [95]:
#previous CH3F model. This should be what the flipped models look like except for one (or a group of dups) that are flipped. 

#original CH3F based on David's general input.py setup 
CH3F_chmkn = full_path + 'projects/halogens/refrigerants/blends/redos_for_march_conference/singles/CH3F/chemkin/chem_annotated.inp'
CH3F_transport = full_path + 'projects/halogens/refrigerants/blends/redos_for_march_conference/singles/CH3F/chemkin/tran.dat'
CH3F_species = full_path + 'projects/halogens/refrigerants/blends/redos_for_march_conference/singles/CH3F/chemkin/species_dictionary.txt'

#load previous CH3F
species_CH3F, reactions_CH3F = load_chemkin_file(CH3F_chmkn, dictionary_path=CH3F_species, transport_path=CH3F_transport)


#this is one of the new flipped models 
flipped_chmkn = full_path + 'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/flipping/flipped_models/376.inp'

#load flipped
species_flipped, reactions_flipped = load_chemkin_file(flipped_chmkn, dictionary_path=CH3F_species, transport_path=CH3F_transport)


#comparison of flipped and previous CH3F model 
model_1_species_list=species_flipped
model_2_species_list=species_CH3F
model_1_reactions= reactions_flipped
model_2_reactions= reactions_CH3F


similar_kinetics_reactions, identical_kinetics_reactions, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactions_flipped, unique_reactions_previous = compare_rxns(model_1_reactions, model_2_reactions)
common_species, unique_species_flipped, unique_species_previous = compare_species(model_1_species_list, model_2_species_list)


1440 common reactions
83 common species


In [96]:
len(common_reactions), len(model_2_reactions), len(model_1_reactions), len(unique_reactions_previous), len(unique_reactions_flipped)

(1440, 1440, 1440, 0, 0)

In [108]:
models = [file for file in os.listdir('./flipped_models') if '.inp' in file]
for model in models:
    flipped_chmkn = full_path + f'projects/PFAS/models/testing_Siddhas_chemistry/CH3F/flipping/flipped_models/{model}'
    species_flipped, reactions_flipped = load_chemkin_file(flipped_chmkn, dictionary_path=CH3F_species, transport_path=CH3F_transport)
    model_1_species_list=species_flipped
    model_1_reactions= reactions_flipped
    similar_kinetics_reactions, identical_kinetics_reactions, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactions_flipped, unique_reactions_previous = compare_rxns(model_1_reactions, model_2_reactions)

    total=0
    for i in reactions_CH3F: 
        for j in reactions_flipped:
            if i.is_isomorphic(j) and i.kinetics.is_identical_to(j.kinetics):
                total+=1
                break
    if total!=1440 and total!=1439 and total!=1438:
        print(model, total)
    
###YESSSSS that means only one was flipped (gives answer 1439). Check with some that have 2 flipped, should give answer 1438 (but might not if kinetics werent different)

1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions
1440 common reactions


KeyboardInterrupt: 

In [118]:
s=[x.replace('.inp','.cti') for x in os.listdir('./flipped_models') if '.inp' in x]
j = [f'copy_{x}' for x in s]
len(j)

322